# 9-2. Semantic Segmentation vs Instance Segmentation

## Segmentation에 대한 이해

<img src="https://drive.google.com/uc?id=1wZlBAkJSRbkrjfVHBV8YCKIp77Mkhwt8">

[segmentation]

https://towardsdatascience.com/review-deepmask-instance-segmentation-30327a072339

segmentation은 이미지를 픽셀 단위로 나누어서 특정 픽셀이 무엇을 지칭하는지를 파악하는 것입니다.


## Semantic Segmentation vs. Instance Segmentation

- semantic segmentation: 하나의 이미지 안에 들어있는 객체의 종류(object category)를 픽셀 단위로 찾습니다.
- instance segmentation: 하나의 이미지 안에 들어있는 객체의 개체(object instance)를 픽셀 단위로 찾습니다.

instance segmentation = semantic segmentation + "distinguishing instances"

이미지 데이터를 segmentation 모델 학습에 사용하기 위해서는 픽셀 하나하나 labeling을 해줘야 하기 때문에 데이터셋 구축이 어렵고, 따라서 data augmentation이 매우 중요합니다.

# 9-3. U-Net 구조를 통해서 Segmentation 이해하기

## Semantic segmentation의 목표

semantic segmentation의 목표는 이미지가 주어졌을 때, 픽셀 단위로 classification을 수행하여 이미지와 동일한 높이와 너비를 가진 segmentation map을 생성하는 것입니다. (아래 사진의 경우 5개의 label 존재)

<img src="https://drive.google.com/uc?id=1IE19oxhNb76cFAEKevBNBhGU_DzJNP-H">

[semantic segmentation]

https://www.jeremyjordan.me/semantic-segmentation/


<img src="https://drive.google.com/uc?id=1eXfl_yyTMmcpTCMt395jQbgMqNSTkhps">

[semantic segmentation]

https://www.jeremyjordan.me/semantic-segmentation/

N개의 클래스에 대해서 semantic segmentation을 진행한다면 N개의 segmentation map(각각은 binary encoding)이 만들어져야 합니다.


<img src="https://drive.google.com/uc?id=1N6cQ2vKF82hjAVcDpIWj4slw1LZVxJa6">

https://dacon.io/competitions/official/235591/overview/description

output이 실수값이라면 regression task입니다. 기본적으로 segmentation은 분류 태스크이지만 segmentation 모델을 통해서 regression 태스크도 수행할 수 있습니다. 즉 위와 같은 인공위성 데이터를 이용한 강우량 예측은 semantic segmentation이지만 regression입니다. 이에 대한 자세한 사항은 링크를 통해 알아보세요.

## U-Net 모델 대략적으로 살펴보기

- U-Net: Convolutional Networks for Biomedical Image Segmentation

### Encoder-Decoder 모델


<img src="https://drive.google.com/uc?id=1hxs4waBKx3A2wmBCc0l1PbXS1tgrfwop">

[Encoder-Decoder 모델]

https://hackernoon.com/autoencoders-deep-learning-bits-1-11731e200694

- Encoder와 Decoder로 구성되어 있습니다.
- 이전 노드에서 배웠던 Transposed Convolution을 사용합니다.
- 입력 이미지와 결과값 모두 3개의 채널을 가집니다. (RGB)
- Encoder에서는 convolution 연산을 수행하고, Decoder에서는 transposed convolution 연산을 수행합니다.
- U-Net: Encoder-Decoder 모델에 skip connection을 추가한 모델!

## U-Net 모델 속 연산 확인하기

### Contracting path

- 3 x 3 convolution 2번

- Padding 사용하지 않음

- ReLU activation

- 2 x 2 max pooling

- Down sampling 후 convolution channel size가 2배

### Expanding (Expansive) path

- 2 x 2 up-convolution
- up-convolution (up-sampling) 후 channel의 수가 1/2이 되고, feature map의 size는 늘어남
- Cropped된 feature map과 concatenation
- 마지막 layer에 1x1 convolution 연산
자세한 설명은 영상을 통해 확인해 보세요. 🙂

<img src = "https://drive.google.com/uc?id=1pcAwlXnZ6lVwKDkN3UYY3HEiuAB2g6HV">

https://arxiv.org/pdf/1505.04597.pdf

### Contracting path

convolution 연산으로 이루어진 부분 (위 그림의 왼쪽 절반, Encoder에 해당됩니다.)

- CNN 구조와 유사합니다.
- 3x3 kernel을 사용하는 VGG 모델과 매우 유사합니다.
- 입력 이미지가 가지고 있는 context 정보를 추출합니다.
- 이미지의 위치에 대한 정보가 차츰 사라집니다.

### Expanding (Expansive) path

up-convolution 연산으로 이루어진 부분 (위 그림의 오른쪽 절반, Decoder에 해당됩니다.)

- low resolution의 latent representation을 high resolution으로 변형합니다.
- contracting path에서 만들어진 feature map을 cropping한 결과물이 concatenation 됩니다.
- 원본 이미지가 가지고 있었던 위치 정보가 복원됩니다.

### Skip connection

- decoding 단계에서, 저차원의 정보와 고차원의 정보도 함께 이용하기 위한 방법입니다.
- encoding 과정에서의 정보 손실을 보충합니다.

# 9-4. U-Net 코드를 통해서 이해 다지기

## U-Net 코드 살펴보기

U-Net 모델의 코드를 보면서 구조를 이해해봅시다.

먼저 시각화에 필요한 라이브러리들을 설치하고 필요한 라이브러리를 불러옵니다.

In [1]:
!pip install graphviz
!pip install pydot

In [2]:
import tensorflow.keras.layers as layers
import tensorflow as tf

U-Net 모델을 구현한 코드입니다.

<img src="https://drive.google.com/uc?id=1VkKkoJo2W3RqfhX9WlObh0nuVLdPPGHm">

[Contracting Path]


<img src="https://drive.google.com/uc?id=1VkKkoJo2W3RqfhX9WlObh0nuVLdPPGHm">

[Expanding Path]



In [3]:
inputs = layers.Input(shape=(572, 572, 1))

# Contracting path 시작
# [1]
conv0 = layers.Conv2D(64, activation='relu', kernel_size = 3)(inputs)
conv1 = layers.Conv2D(64, activation='relu', kernel_size=3)(conv0)  # Skip connection으로 Expanding path로 이어질 예정
conv2 = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(conv1)

# [2]
conv3 = layers.Conv2D(128, activation='relu', kernel_size=3)(conv2)
conv4 = layers.Conv2D(128, activation='relu', kernel_size=3)(conv3)  # Skip connection으로 Expanding path로 이어질 예정
conv5 = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(conv4)

# [3]
conv6 = layers.Conv2D(256, activation='relu', kernel_size=3)(conv5)
conv7 = layers.Conv2D(256, activation='relu', kernel_size=3)(conv6)  # Skip connection으로 Expanding path로 이어질 예정
conv8 = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(conv7)

# [4]
conv9 = layers.Conv2D(512, activation='relu', kernel_size=3)(conv8)
conv10 = layers.Conv2D(512, activation='relu', kernel_size=3)(conv9)  # Skip connection으로 Expanding path로 이어질 예정
conv11 = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(conv10)

# [5]
conv12 = layers.Conv2D(1024, activation='relu', kernel_size=3)(conv11)
conv13 = layers.Conv2D(1024, activation='relu', kernel_size=3)(conv12)
# Contracting path 끝

# Expanding path 시작
# [6]
trans01 = layers.Conv2DTranspose(512, kernel_size=2, strides=(2, 2), activation='relu')(conv13)
crop01 = layers.Cropping2D(cropping=(4, 4))(conv10)
concat01 = layers.concatenate([trans01, crop01], axis=-1)

# [7]
conv14 = layers.Conv2D(512, activation='relu', kernel_size=3)(concat01)
conv15 = layers.Conv2D(512, activation='relu', kernel_size=3)(conv14)
trans02 = layers.Conv2DTranspose(256, kernel_size=2, strides=(2, 2), activation='relu')(conv15)

# [8]
crop02 = layers.Cropping2D(cropping=(16, 16))(conv7)
concat02 = layers.concatenate([trans02, crop02], axis=-1)

# [9]
conv16 = layers.Conv2D(256, activation='relu', kernel_size=3)(concat02)
conv17 = layers.Conv2D(256, activation='relu', kernel_size=3)(conv16)
trans03 = layers.Conv2DTranspose(128, kernel_size=2, strides=(2, 2), activation='relu')(conv17)

# [10]
crop03 = layers.Cropping2D(cropping=(40, 40))(conv4)
concat03 = layers.concatenate([trans03, crop03], axis=-1)

# [11]
conv18 = layers.Conv2D(128, activation='relu', kernel_size=3)(concat03)
conv19 = layers.Conv2D(128, activation='relu', kernel_size=3)(conv18)
trans04 = layers.Conv2DTranspose(64, kernel_size=2, strides=(2, 2), activation='relu')(conv19)

# [12]
crop04 = layers.Cropping2D(cropping=(88, 88))(conv1)
concat04 = layers.concatenate([trans04, crop04], axis=-1)

# [13]
conv20 = layers.Conv2D(64, activation='relu', kernel_size=3)(concat04)
conv21 = layers.Conv2D(64, activation='relu', kernel_size=3)(conv20)
# Expanding path 끝
outputs = layers.Conv2D(2, kernel_size=1)(conv21)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="u-netmodel")

In [4]:
model.summary()

Model: "u-netmodel"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 572, 572, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 570, 570, 64)         640       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 568, 568, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 284, 284, 64)         0         ['conv2d_1[0][0]']            
 D)                                                                                      

위에서 구현한 U-Net 모델의 구조를 그림으로 나타낼 수 있습니다.

In [6]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes= True, show_layer_names=True, dpi=80).create(prog='dot', format='svg'))  #dpi를 작게 하면 그래프가 커집니다.

ModuleNotFoundError: No module named 'keras.utils.vis_utils'

## Transposed Convolution을 코드로 살펴보기

Transposed Convolution을 코드를 통해 살펴 봅시다. 먼저 필요한 모듈을 불러옵니다.

In [7]:
# 필요한 모듈 불러오기
import numpy as np
import tensorflow as tf

input 데이터를 만들고 모델에 맞게 reshaping합니다.

In [8]:
# input data
X = np.asarray([[1, 2],
			  [3, 4]])

In [9]:
print(X)
print(X.shape)

[[1 2]
 [3 4]]
(2, 2)


In [10]:
# 모델에 맞게 reshaping
X = X.reshape((1, 2, 2, 1))

In [11]:
print(X)
print(X.shape)

[[[[1]
   [2]]

  [[3]
   [4]]]]
(1, 2, 2, 1)


Transposed convolution 모델을 만들어 봅시다. 커널 사이즈 중 (1, 1) 부분을 다양하게 바꿔 보세요.

In [12]:
# 모델 만들기
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2DTranspose(1, (1, 1), strides=(2, 2), input_shape=(2, 2, 1))) # Conv2DTranspos layer
# 다양한 커널 사이즈로 바꿔보세요.

weights를 설정하고 설정한 weights를 모델에 적용합니다.

In [13]:
weights = [np.asarray([[[[1]]]]), np.asarray([1])] # weight = 1, bias = 1

In [14]:
weights

[array([[[[1]]]]), array([1])]

In [15]:
model.set_weights(weights)

In [16]:
yhat = model.predict(X)
yhat = yhat.reshape((4, 4)) # 결과를 확인하기 편하게 reshaping
print(yhat)

1/1 [==============================] - 0s 211ms/step
[[2. 1. 3. 1.]
 [1. 1. 1. 1.]
 [4. 1. 5. 1.]
 [1. 1. 1. 1.]]


## 학습정리


- Semantic segmentation은 이미지의 픽셀 단위별로 무엇을 지칭하는지를 판단(classification)하는 task입니다.
- Semantic segmentation에서 instance(개체)를 구분하는 것까지 추가된 것이 instance segmentation입니다.
- U-Net 모델은 Encoder-Decoder 모델에 skip connection을 추가한 모델입니다.